In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_excel('массив 1 класс.xlsx')

In [3]:
df.columns = ['Услуга', 'УФПС приема', 'УФПС назначения', 'Тип пересылки', 'Брейк',
       'ОЦ', 'cnt18', 'revenue18', 'ОЦ18', 'тариф ОЦ18', 'cnt', 'revenue',
       'ОЦ19', 'тариф ОЦ']

In [4]:
df['цена0'] = df['revenue18'] / df['cnt18']
df['цена1'] = df['revenue'] / df['cnt']

df.loc[df['цена0'].isna(),'цена0'] = df.loc[df['цена0'].isna(),'цена1']
df.loc[df['цена1'].isna(),'цена1'] = df.loc[df['цена1'].isna(),'цена0']

#df = df[~df['цена1'].isna()]

df['выручка итог 0'] = df['revenue18'] + (df['тариф ОЦ18'].astype(int))/100
df['выручка итог 1'] = df['revenue'] + (df['тариф ОЦ'].astype(int))/100

summary = pd.DataFrame(df.iloc[:,6:].sum()).T

In [5]:
# делаем разбивку по долям ГЕО СПЛИТ
geo_0 = df[['УФПС приема', 'УФПС назначения', 'cnt18']].groupby(['УФПС приема', 'УФПС назначения']).sum()/summary.loc[0,'cnt18']
geo_0.reset_index(inplace = True)
geo_0['link'] = geo_0['УФПС приема'] + geo_0['УФПС назначения'].astype(str)
geo_0_dict = geo_0[['link','cnt18']].set_index('link').to_dict()['cnt18']

geo_1 = df[['УФПС приема', 'УФПС назначения', 'cnt']].groupby(['УФПС приема', 'УФПС назначения']).sum()/summary.loc[0,'cnt']
geo_1.reset_index(inplace = True)
geo_1['link'] = geo_1['УФПС приема'] + geo_1['УФПС назначения'].astype(str)
geo_1_dict = geo_1[['link','cnt']].set_index('link').to_dict()['cnt']

In [6]:
# делаем разбивку по БРЕЙКАМ
break_0 = df[['УФПС приема', 'УФПС назначения','Брейк','cnt18']].groupby(['УФПС приема', 'УФПС назначения','Брейк']).sum().reset_index()
break_0['cnt geo'] =  (break_0['УФПС приема'] + break_0['УФПС назначения'].astype(str)).map(geo_0_dict) * summary.loc[0,'cnt18']
break_0['break_part'] = break_0['cnt18'] / break_0['cnt geo']
break_0['link'] = break_0['УФПС приема'] + break_0['УФПС назначения'].astype(str) + break_0['Брейк'].astype(str)
break_0_dict = break_0[['link','break_part']].set_index('link').to_dict()['break_part']

break_1 = df[['УФПС приема', 'УФПС назначения','Брейк','cnt']].groupby(['УФПС приема', 'УФПС назначения','Брейк']).sum().reset_index()
break_1['cnt geo'] =  (break_1['УФПС приема'] + break_1['УФПС назначения'].astype(str)).map(geo_1_dict) * summary.loc[0,'cnt']
break_1['break_part'] = break_1['cnt'] / break_1['cnt geo']
break_1['link'] = break_1['УФПС приема'] + break_1['УФПС назначения'].astype(str) + break_1['Брейк'].astype(str)
break_1_dict = break_1[['link','break_part']].set_index('link').to_dict()['break_part']

In [7]:
# делаем разбивку по ОЦ
oc_0 = df[['УФПС приема', 'УФПС назначения','Брейк','ОЦ','cnt18']].groupby(['УФПС приема', 'УФПС назначения','Брейк', 'ОЦ']).sum().reset_index()
oc_0['link'] = oc_0['УФПС приема'] + oc_0['УФПС назначения'].astype(str) + oc_0['Брейк'].astype(str)
oc_0['cnt_break'] = oc_0['link'].map(break_0[['link','cnt18']].set_index('link').to_dict()['cnt18'])
oc_0['oc_dolya'] = oc_0['cnt18'] / oc_0['cnt_break']
oc_0['link'] = oc_0['link'] + oc_0['ОЦ']
oc_0_dict = oc_0[['link','oc_dolya']].set_index('link').to_dict()['oc_dolya']


oc_1 = df[['УФПС приема', 'УФПС назначения','Брейк','ОЦ','cnt']].groupby(['УФПС приема', 'УФПС назначения','Брейк', 'ОЦ']).sum().reset_index()
oc_1['link'] = oc_1['УФПС приема'] + oc_1['УФПС назначения'].astype(str) + oc_1['Брейк'].astype(str)
oc_1['cnt_break'] = oc_1['link'].map(break_1[['link','cnt']].set_index('link').to_dict()['cnt'])
oc_1['oc_dolya'] = oc_1['cnt'] / oc_1['cnt_break']
oc_1['link'] = oc_1['link'] + oc_1['ОЦ']
oc_1_dict = oc_1[['link','oc_dolya']].set_index('link').to_dict()['oc_dolya']

In [8]:
# подтягиваем все доли к исходной таблице
df['доля направления 0'] = (df['УФПС приема'] + df['УФПС назначения'].astype(str)).map(geo_0_dict).fillna(0)
df['доля типа пересылки 0'] = 1
df['доля break 0'] = (df['УФПС приема'] + df['УФПС назначения'].astype(str) + df['Брейк'].astype(str)).map(break_0_dict).fillna(0)
df['доля оц 0'] = (df['УФПС приема'] + df['УФПС назначения'].astype(str) + df['Брейк'].astype(str) + df['ОЦ']).map(oc_0_dict).fillna(0)

df['доля направления 1'] = (df['УФПС приема'] + df['УФПС назначения'].astype(str)).map(geo_1_dict).fillna(0)
df['доля типа пересылки 1'] = 1
df['доля break 1'] = (df['УФПС приема'] + df['УФПС назначения'].astype(str) + df['Брейк'].astype(str)).map(break_1_dict).fillna(0)
df['доля оц 1'] = (df['УФПС приема'] + df['УФПС назначения'].astype(str) + df['Брейк'].astype(str) + df['ОЦ']).map(oc_1_dict).fillna(0)

In [9]:
df['фактор объемов'] = summary.loc[0,'cnt'] * df['доля направления 0'] * df['доля типа пересылки 0'] * df['доля break 0'] * df['доля оц 0'] * df['цена0'] - df['revenue18']

df['фактор цены'] = summary.loc[0,'cnt'] * df['доля направления 0'] * df['доля типа пересылки 0'] * df['доля break 0'] * df['доля оц 0'] * df['цена1'] - df['revenue18']

df['фактор гео'] = summary.loc[0,'cnt'] * df['доля направления 1'] * df['доля типа пересылки 0'] * df['доля break 0'] * df['доля оц 0'] * df['цена1'] - df['revenue18']
df.loc[df['доля направления 0']==0,'фактор гео'] = df.loc[df['доля направления 0']==0,'revenue']

df['фактор тип пересылки'] = summary.loc[0,'cnt'] * df['доля направления 1'] * df['доля типа пересылки 1'] * df['доля break 0'] * df['доля оц 0'] * df['цена1'] - df['revenue18']
df.loc[df['доля направления 0']==0,'фактор тип пересылки'] = df.loc[df['доля направления 0']==0,'revenue']

df['фактор брейка'] = summary.loc[0,'cnt'] * df['доля направления 1'] * df['доля типа пересылки 1'] * df['доля break 1'] * df['доля оц 0'] * df['цена1'] - df['revenue18']
df.loc[df['доля break 0']==0,'фактор брейка'] = df.loc[df['доля break 0']==0,'revenue']

df['фактор ОЦ'] = summary.loc[0,'cnt'] * df['доля направления 1'] * df['доля типа пересылки 1'] * df['доля break 1'] * df['доля оц 1'] * df['цена1'] - df['revenue18']
df.loc[df['доля оц 0']==0,'фактор ОЦ'] = df.loc[df['доля оц 0']==0,'revenue']

In [10]:
factor_sum = pd.DataFrame(df[['фактор объемов','фактор цены','фактор гео','фактор тип пересылки','фактор брейка', 'фактор ОЦ']].sum()).T
factor_sum


,фактор объемов,фактор цены,фактор гео,фактор тип пересылки,фактор брейка,фактор ОЦ
0,1.172064e+08,2.034842e+08,2.009515e+08,2.009515e+08,2.182770e+08,2.047266e+08


In [11]:
sdt_by_factors = pd.DataFrame(columns = ['std0',  'объемы', 'цена', 'гео', 'тип пересылки', 'брейк',  'оц', 'sdt1'],index = [0])
sdt_by_factors['std0'] = summary.loc[0,'revenue18'] / summary.loc[0,'cnt18']
sdt_by_factors['объемы'] = (summary.loc[0,'revenue18'] + factor_sum.loc[0,'фактор объемов']) / summary.loc[0,'cnt']
sdt_by_factors['цена'] = (summary.loc[0,'revenue18']  + factor_sum.loc[0,'фактор цены']) / summary.loc[0,'cnt']
sdt_by_factors['гео'] = (summary.loc[0,'revenue18']  + factor_sum.loc[0,'фактор гео']) / summary.loc[0,'cnt']
sdt_by_factors['тип пересылки'] = (summary.loc[0,'revenue18']  + factor_sum.loc[0,'фактор тип пересылки']) / summary.loc[0,'cnt']
sdt_by_factors['брейк'] = (summary.loc[0,'revenue18']  + factor_sum.loc[0,'фактор брейка']) / summary.loc[0,'cnt']
sdt_by_factors['оц'] = (summary.loc[0,'revenue18']  + factor_sum.loc[0,'фактор ОЦ']) / summary.loc[0,'cnt']
sdt_by_factors['sdt1'] = summary.loc[0,'revenue'] / summary.loc[0,'cnt']
sdt_by_factors = sdt_by_factors.round(2)

In [12]:
sdt_by_factors.loc['delta',:] = 0
for i in range(1,len(sdt_by_factors.T)):
    sdt_by_factors.iloc[:,i]['delta'] = sdt_by_factors.iloc[:,i][0] - sdt_by_factors.iloc[:,i-1][0]


sdt_by_factors

,std0,объемы,цена,гео,тип пересылки,брейк,оц,sdt1
0,225.74,225.74,280.15,278.56,278.56,289.48,280.94,280.94
delta,0.00,0.00,54.41,-1.59,0.00,10.92,-8.54,0.00


In [14]:
df.to_excel('ФА проверка скрипта.xlsx', index = False)